# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import json
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
outputfile = "../output_data/cities.csv"
weather_data_df = pd.read_csv(outputfile)
weather_data_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Whyalla,-33.03,137.58,62.60,72,90,14.99,AU,1604112806
1,1,Poum,-20.23,164.02,79.02,76,99,19.89,NC,1604112806
2,2,Provideniya,64.38,-173.30,21.97,84,20,7.25,RU,1604112806
3,3,Baruun-Urt,46.68,113.28,36.99,70,29,19.10,MN,1604112806
4,4,Skjervøy,70.03,20.97,32.27,87,0,18.54,NO,1604112806
...,...,...,...,...,...,...,...,...,...,...
574,574,Mattawa,46.74,-119.90,50.00,43,1,5.82,US,1604113311
575,575,Kichera,55.94,110.10,24.49,87,100,3.13,RU,1604113311
576,576,Warwick,41.70,-71.42,32.00,92,1,4.70,US,1604113311
577,577,Beauceville,46.22,-70.78,23.00,85,19,1.34,CA,1604113311


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Access maps with unique API key
gmaps.configure(api_key=g_key)

In [6]:
locations = weather_data_df[["Lat", "Lng"]]
humidity = weather_data_df['Humidity']
figure_layout = {
    'width': '400px',
    'height': '700px',
    'border': '1px solid black',
    'padding': '3px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig = gmaps.figure(center=(46.0, -5.0), zoom_level=1)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
max_humidity = max(humidity)
heat_layer.max_intensity = max_humidity
heat_layer.point_radius = 2
#heat_layer.gradient = ['white', 'red']

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [9]:
perfect_weather = weather_data_df.loc[(weather_data_df['Max Temp'] > 70) & (weather_data_df['Max Temp']  <= 82) & 
                                      (weather_data_df['Cloudiness'] <5) & 
                                      (weather_data_df['Wind Speed'] <10) ]

perfect_weather.drop(perfect_weather.columns[1], axis=1)
perfect_weather = perfect_weather.drop(perfect_weather.columns[0], axis=1)
perfect_weather = perfect_weather.reset_index()
perfect_weather = perfect_weather.dropna()
perfect_weather

,index,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,34,Rikitea,-23.12,-134.97,76.15,63,0,0.98,PF,1604112348
1,65,Binga,-17.62,27.34,73.45,45,0,1.79,ZW,1604112853
2,67,Miandrivazo,-19.52,45.47,77.85,41,0,5.48,MG,1604112854
3,101,Airai,-8.93,125.41,80.38,39,0,4.00,TL,1604112897
4,109,Guaratinga,-16.57,-39.57,77.00,83,0,8.05,BR,1604112898
5,170,Umm Kaddadah,13.60,26.69,79.03,23,0,7.16,SD,1604112944
6,177,Satānā,20.58,74.20,75.63,43,0,3.24,IN,1604112945
7,178,Alice Springs,-23.70,133.88,77.00,17,0,6.93,AU,1604112923
8,220,Salalah,17.02,54.09,73.40,40,0,3.36,OM,1604112992
9,252,Verāval,20.90,70.37,80.96,48,0,7.58,IN,1604113036


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [10]:
hotel_df = pd.DataFrame({'Hotel Name': "",
                        'City': perfect_weather['City'],
                        'Country': perfect_weather['Country'],
                         'Lat':perfect_weather['Lat'],
                         'Lng':perfect_weather['Lng']
                        }
                       )
hotel_df

,Hotel Name,City,Country,Lat,Lng
0,,Rikitea,PF,-23.12,-134.97
1,,Binga,ZW,-17.62,27.34
2,,Miandrivazo,MG,-19.52,45.47
3,,Airai,TL,-8.93,125.41
4,,Guaratinga,BR,-16.57,-39.57
5,,Umm Kaddadah,SD,13.60,26.69
6,,Satānā,IN,20.58,74.20
7,,Alice Springs,AU,-23.70,133.88
8,,Salalah,OM,17.02,54.09
9,,Verāval,IN,20.90,70.37


In [11]:
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "hotels",
    "keyword": "motel",
    "key": g_key
}
# Use the lat/lng we recovered to identify cities with perfect weather
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

     # add location each iteration 
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "4 star" and our lat/lng ( I ran a search with lat amd lng the results were bad)
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
#   print the name_address url, avoid doing for public github repos in order to avoid exposing key
    #print(name_address.url)

    # convert to json
    name_address = name_address.json()
    #print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]['name']
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")


Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


In [12]:
hotel_df

,Hotel Name,City,Country,Lat,Lng
0,Pension Maro'i,Rikitea,PF,-23.12,-134.97
1,,Binga,ZW,-17.62,27.34
2,Princesse Tsiribihina,Miandrivazo,MG,-19.52,45.47
3,,Airai,TL,-8.93,125.41
4,Hotel Caraíva,Guaratinga,BR,-16.57,-39.57
5,,Umm Kaddadah,SD,13.60,26.69
6,,Satānā,IN,20.58,74.20
7,The Diplomat Motel,Alice Springs,AU,-23.70,133.88
8,Alpha lodge,Salalah,OM,17.02,54.09
9,Hotel Madhuram,Verāval,IN,20.90,70.37


In [13]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [14]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,info_box_content=hotel_info)
# Add the layer to the map
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))